# **Важно!** 

Домашнее задание состоит из нескольких задач, которые вам нужно решить.
*   Баллы выставляются по принципу выполнено/невыполнено.
*   За каждую выполненую задачу вы получаете баллы (количество баллов за задание указано в скобках).

**Инструкция выполнения:** Выполните задания в этом же ноутбуке (места под решения **КАЖДОЙ** задачи обозначены как **#НАЧАЛО ВАШЕГО РЕШЕНИЯ** и **#КОНЕЦ ВАШЕГО РЕШЕНИЯ**)

**Как отправить задание на проверку:** Вам необходимо сохранить ваше решение в данном блокноте и отправить итоговый **файл .IPYNB** в личном сообщении Telegram.

# **Прежде чем проверять задания:**

1. Перезапустите **ядро (restart the kernel)**: в меню, выбрать **Ядро (Kernel)**
→ **Перезапустить (Restart)**
2. Затем **Выполнить** **все ячейки (run all cells)**: в меню, выбрать **Ячейка (Cell)**
→ **Запустить все (Run All)**.

# Домашнее задание — LangChain и инференс

Цель: перевести домашнее задание на использование LangChain. В задании — 4 задачи, в том числе про LCEL и Structured Output. Для каждой задачи дан стартовый код.

---

### Задачи (кратко)

1. Task 1 — Быстрый старт LangChain: Prompt + LLM
2. Task 2 — Chains и составные сценарии
3. Task 3 — LCEL: сценарий с несколькими шагами логики
4. Task 4 — Structured Output: схемы и валидация

Дальше следует подробное описание каждой задачи с критериями и стартовым кодом.

In [1]:
# Установка зависимостей (выполните в среде развертывания один раз)
# !pip install langchain openai pydantic

OPENROUTER_API_KEY =  "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"


---

## Task 1 — Quick LangChain prompt

Цель: собрать минимальную LCEL-цепочку «prompt → LLM → парсер», которая возвращает краткий ответ модели на запрос "Механизм внимания".

Acceptance criteria:
- Используется `ChatPromptTemplate` и `ChatOpenAI` (или совместимая чат-модель), соединённые через LCEL (`|`).
- В цепочку добавлен `StrOutputParser` и приведён пример вызова `chain.invoke(...)`.
- В примере показано, где брать API-ключ (например, из переменной окружения `OPENROUTER_API_KEY`).

Starter code:

```python
import os

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "Ты отвечаешь кратко и по делу на русском языке."),
    ("human", "Объясни тему: {topic}"),
])

llm = ChatOpenAI(
    model="openai/gpt-5-nano",
    temperature=0,
    api_key=os.getenv("OPENROUTER_API_KEY", OPENROUTER_API_KEY),
    base_url="https://openrouter.ai/api/v1",
)

chain = prompt | llm | StrOutputParser()

response = chain.invoke({"topic": "LSTM"})
print(response)
```


In [2]:
# Task 1 — Quick LangChain prompt
# НАЧАЛО ВАШЕГО РЕШЕНИЯ

# КОНЕЦ ВАШЕГО РЕШЕНИЯ

## Task 2 — Chains: составной pipeline

Цель: собрать LCEL-пайплайн из двух шагов: (1) LLM подбирает факты по теме `"transformers"`, (2) отдельный шаг формирует резюме из заметок.

Acceptance criteria:
- Используется LangChain Expression Language (композиции `|`, `RunnablePassthrough`, `RunnableLambda`) вместо `SequentialChain`. `RunnablePassthrough` просто прокидывает вход дальше по цепочке без изменений. В LCEL его используют, когда нужно передать исходные данные как часть словаря в последующие шаги: например, `{"topic": RunnablePassthrough(), "research_notes": research_chain}` — так и оригинальная тема, и результаты ресёрча доступны в следующем промпте.
- Шаблоны prompt'ов для ресёрча и итогового резюме разделены и связаны логикой.
- Есть пример вызова `.invoke` c входным топиком.

Starter code:

```python
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="openai/gpt-5-nano",
    temperature=0,
    api_key=os.getenv("OPENROUTER_API_KEY", OPENROUTER_API_KEY),
    base_url="https://openrouter.ai/api/v1",
)

research_prompt = ChatPromptTemplate.from_messages([
    ("system", "Собери три факта по теме."),
    ("human", "Тема: {topic}"),
])
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "Сделай короткое резюме на русском."),
    ("human", "Суммаризируй заметки: {research_notes}"),
])

research_chain = research_prompt | llm | StrOutputParser()

pipeline = (
    {
        "topic": RunnablePassthrough(),
        "research_notes": research_chain,
    }
    | summary_prompt
    | llm
    | StrOutputParser()
)

response = pipeline.invoke("rnns")
print(response)
```


In [3]:
# Task 2 — Исследование + резюме в одном chain
# НАЧАЛО ВАШЕГО РЕШЕНИЯ

# КОНЕЦ ВАШЕГО РЕШЕНИЯ


## Task 3 — LCEL: сложная логика в несколько шагов

Цель: построить условный роутер запросов на LCEL с несколькими ветками и fallback.
1 Дать определение диффузии и 2 сравнить RNN и Transformer

Acceptance criteria:
- Предусмотрена классификация интента (`RunnableLambda` добавляет ключ `intent`).
- `RunnableBranch` маршрутизирует запрос в разные цепочки и содержит запасной путь.
- Показан пример вызова `.invoke`, демонстрирующий выбранную ветку.

Starter code / схема:

```python
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableBranch, RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="openai/gpt-5-nano",
    temperature=0,
    api_key=os.getenv("OPENROUTER_API_KEY", OPENROUTER_API_KEY),
    base_url="https://openrouter.ai/api/v1",
)

definition_chain = (
    ChatPromptTemplate.from_template("Дай определение: {question}")
    | llm
    | StrOutputParser()
)
comparison_chain = (
    ChatPromptTemplate.from_template("Сравни подходы: {question}")
    | llm
    | StrOutputParser()
)

def classify(inputs):
    question = inputs["question"].lower()
    if "сравн" in question:
        intent = "comparison"
    elif "что такое" in question or "определ" in question:
        intent = "definition"
    else:
        intent = "fallback"
    return {"intent": intent, **inputs}

router = RunnableBranch(
    (lambda data: data["intent"] == "comparison", comparison_chain),
    (lambda data: data["intent"] == "definition", definition_chain),
    RunnableLambda(lambda data: "Не знаю, уточните запрос."),
)

workflow = (
    {"question": RunnablePassthrough()}
    | RunnableLambda(classify)
    | router
)
response1 = workflow.invoke("Что такое LSTM?")


print(response1)

response2 = workflow.invoke("Сравни TF-IDF и BoW")

print(response2)
```


In [4]:
# Task 3 — LCEL: сложная логика в несколько шагов
# НАЧАЛО ВАШЕГО РЕШЕНИЯ

# КОНЕЦ ВАШЕГО РЕШЕНИЯ


## Task 4 — Structured Output

Цель: использовать актуальный `PydanticOutputParser` для строгой схемы ответа для темы `LLM`.

Acceptance criteria:
- Описана Pydantic-модель с типами и описаниями полей.
- Prompt включает `parser.get_format_instructions()` и цепочка вызывает `.invoke`.
- Полученный результат приводится к `dict`/`BaseModel` и выводится.

Starter code:

```python
from pydantic import BaseModel, Field

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

class Fact(BaseModel):
    title: str = Field(..., description="Короткий заголовок")
    summary: str
    confidence: float

class FactCollection(BaseModel):
    topic: str
    facts: list[Fact]

parser = PydanticOutputParser(pydantic_object=FactCollection)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Формируй JSON со списком фактов."),
    ("human", "Тема: {topic}\nФормат: {format_instructions}"),
]).partial(format_instructions=parser.get_format_instructions())

llm = ChatOpenAI(
    model="openai/gpt-5-nano",
    temperature=0,
    api_key=os.getenv("OPENROUTER_API_KEY", OPENROUTER_API_KEY),
    base_url="https://openrouter.ai/api/v1",
)
structured_chain = prompt | llm | parser

result = structured_chain.invoke({"topic": "Attention"})
print(result)
```


In [5]:
# Task 4 — Structured Output
# НАЧАЛО ВАШЕГО РЕШЕНИЯ

# КОНЕЦ ВАШЕГО РЕШЕНИЯ
